# Chapter 5: Jacobian Computation - Manual vs Automatic

## 🎯 학습 목표

이 챕터에서는 Pose Graph Optimization의 핵심인 Jacobian 계산을 깊이 있게 다룹니다. 수동 계산과 Symforce의 자동 계산을 비교하며 각각의 장단점을 이해합니다.

- Jacobian의 수학적 유도
- 수동 구현의 세부사항과 함정
- Symforce를 통한 자동 Jacobian 생성
- 성능과 정확도 비교
- 실전 최적화 팁

## 📚 이론적 배경

### Between Factor의 Jacobian

Between factor의 에러 함수:
$$e_{ij} = \log(T_{ij}^{meas^{-1}} \cdot T_i^{-1} \cdot T_j)$$

이에 대한 Jacobian:
- $\frac{\partial e_{ij}}{\partial T_i}$: 포즈 i에 대한 미분
- $\frac{\partial e_{ij}}{\partial T_j}$: 포즈 j에 대한 미분

### 왜 정확한 Jacobian이 중요한가?

1. **수렴 속도**: 정확한 Jacobian은 더 빠른 수렴
2. **안정성**: 부정확한 Jacobian은 발산 가능
3. **최종 정확도**: 더 정밀한 최적화 결과

## 🔧 필요한 라이브러리 임포트

In [ ]:
import numpy as np
import symforce
symforce.set_epsilon_to_number()

import symforce.symbolic as sf
from symforce.ops import LieGroupOps
from symforce import codegen
import sym

from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
import time

# 유틸리티 함수들
def rotvec_to_rotmat(rotvec):
    return Rotation.from_rotvec(rotvec).as_matrix()

def rotmat_to_rotvec(R):
    return Rotation.from_matrix(R).as_rotvec()

def rotvec_to_quat(rotvec):
    return Rotation.from_rotvec(rotvec).as_quat()  # [x, y, z, w]

def skew_symmetric(v):
    """벡터를 skew-symmetric 행렬로 변환"""
    return np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

print("✅ 라이브러리 준비 완료!")

## 1. 수동 Jacobian 계산 - 근사 방법

먼저 간단한 근사를 사용한 수동 Jacobian 계산을 구현합니다.

In [ ]:
def compute_between_jacobian_manual_approx(pose_i, pose_j, pose_ij_meas):
    """수동으로 계산한 근사 Jacobian
    
    이 방법은 작은 각도에서만 정확합니다.
    
    Args:
        pose_i, pose_j: {'t': translation, 'r': rotation_vector}
        pose_ij_meas: {'t': translation, 'R': rotation_matrix}
        
    Returns:
        residual: 6D error vector [t_err, r_err]
        Ji, Jj: 6x6 Jacobian matrices
    """
    # 회전 행렬 변환
    Ri = rotvec_to_rotmat(pose_i['r'])
    Rj = rotvec_to_rotmat(pose_j['r'])
    Rij_meas = pose_ij_meas['R']
    
    ti = pose_i['t']
    tj = pose_j['t']
    tij_meas = pose_ij_meas['t']
    
    # 예측된 상대 변환
    Ri_inv = Ri.T
    Rij_pred = Ri_inv @ Rj
    tij_pred = Ri_inv @ (tj - ti)
    
    # 에러 계산
    R_err = Rij_meas.T @ Rij_pred
    t_err = Rij_meas.T @ (tij_pred - tij_meas)
    r_err = rotmat_to_rotvec(R_err)
    
    residual = np.concatenate([t_err, r_err])
    
    # 근사 Jacobian
    Ji = np.zeros((6, 6))
    # Translation 부분
    Ji[:3, :3] = -Rij_meas.T @ Ri_inv  # ∂t_err/∂ti
    Ji[:3, 3:] = Rij_meas.T @ Ri_inv @ skew_symmetric(tj - ti)  # ∂t_err/∂ri
    # Rotation 부분 (근사)
    Ji[3:, 3:] = -np.eye(3)  # ∂r_err/∂ri
    
    Jj = np.zeros((6, 6))
    # Translation 부분
    Jj[:3, :3] = Rij_meas.T @ Ri_inv  # ∂t_err/∂tj
    # Rotation 부분 (근사)
    Jj[3:, 3:] = np.eye(3)  # ∂r_err/∂rj
    
    return residual, Ji, Jj

# 테스트
pose_i = {'t': np.array([1.0, 0.0, 0.0]), 'r': np.array([0.0, 0.0, 0.1])}
pose_j = {'t': np.array([2.0, 0.5, 0.0]), 'r': np.array([0.0, 0.0, 0.3])}
pose_ij_meas = {
    't': np.array([1.0, 0.5, 0.0]),
    'R': rotvec_to_rotmat(np.array([0.0, 0.0, 0.2])),
    'r': np.array([0.0, 0.0, 0.2])
}

res_approx, Ji_approx, Jj_approx = compute_between_jacobian_manual_approx(
    pose_i, pose_j, pose_ij_meas
)

print("📐 근사 Jacobian 결과:")
print(f"Residual: {res_approx}")
print(f"Ji 형태: {Ji_approx.shape}")
print(f"Jj 형태: {Jj_approx.shape}")

## 2. 수동 Jacobian 계산 - 정확한 방법

이제 더 정확한 Jacobian을 유도해봅시다.

In [ ]:
def exp_map_so3(w):
    """SO(3) exponential map: rotation vector → rotation matrix"""
    theta = np.linalg.norm(w)
    if theta < 1e-6:
        return np.eye(3) + skew_symmetric(w)
    else:
        w_hat = skew_symmetric(w)
        return np.eye(3) + (np.sin(theta)/theta) * w_hat + \
               ((1 - np.cos(theta))/theta**2) * w_hat @ w_hat

def log_map_so3(R):
    """SO(3) logarithm map: rotation matrix → rotation vector"""
    trace = np.trace(R)
    if trace >= 3 - 1e-6:
        # Small angle approximation
        return np.array([R[2,1] - R[1,2], 
                        R[0,2] - R[2,0], 
                        R[1,0] - R[0,1]]) / 2
    else:
        theta = np.arccos((trace - 1) / 2)
        return theta / (2 * np.sin(theta)) * np.array([
            R[2,1] - R[1,2],
            R[0,2] - R[2,0], 
            R[1,0] - R[0,1]
        ])

def right_jacobian_so3(w):
    """SO(3)의 right Jacobian"""
    theta = np.linalg.norm(w)
    if theta < 1e-6:
        return np.eye(3) - 0.5 * skew_symmetric(w)
    else:
        w_hat = skew_symmetric(w)
        return np.eye(3) - ((1 - np.cos(theta))/theta**2) * w_hat + \
               ((theta - np.sin(theta))/theta**3) * w_hat @ w_hat

def right_jacobian_inverse_so3(w):
    """SO(3)의 right Jacobian의 역행렬"""
    theta = np.linalg.norm(w)
    if theta < 1e-6:
        return np.eye(3) + 0.5 * skew_symmetric(w)
    else:
        w_hat = skew_symmetric(w)
        cot_half = 1.0 / np.tan(theta / 2)
        return np.eye(3) * (theta / 2) * cot_half + \
               (1 - (theta / 2) * cot_half) * np.outer(w, w) / (theta**2) - \
               0.5 * w_hat

def compute_between_jacobian_manual_exact(pose_i, pose_j, pose_ij_meas):
    """정확한 수동 Jacobian 계산
    
    Lie theory를 사용한 정확한 미분
    """
    # 회전 행렬 변환
    Ri = rotvec_to_rotmat(pose_i['r'])
    Rj = rotvec_to_rotmat(pose_j['r'])
    Rij_meas = pose_ij_meas['R']
    
    ti = pose_i['t']
    tj = pose_j['t']
    tij_meas = pose_ij_meas['t']
    
    # 예측된 상대 변환
    Ri_inv = Ri.T
    Rij_pred = Ri_inv @ Rj
    tij_pred = Ri_inv @ (tj - ti)
    
    # 에러 계산
    R_err = Rij_meas.T @ Rij_pred
    t_err = Rij_meas.T @ (tij_pred - tij_meas)
    r_err = log_map_so3(R_err)
    
    residual = np.concatenate([t_err, r_err])
    
    # Right Jacobian 계산
    Jr_err = right_jacobian_so3(r_err)
    Jr_inv = right_jacobian_inverse_so3(r_err)
    
    # 정확한 Jacobian 계산
    Ji = np.zeros((6, 6))
    
    # Translation 부분
    Ji[:3, :3] = -Rij_meas.T @ Ri_inv
    Ji[:3, 3:] = Rij_meas.T @ Ri_inv @ skew_symmetric(tj - ti)
    
    # Rotation 부분 (정확한 미분)
    # ∂log(R_err)/∂ri = -Jr^{-1}
    Ji[3:, 3:] = -Jr_inv
    
    Jj = np.zeros((6, 6))
    
    # Translation 부분
    Jj[:3, :3] = Rij_meas.T @ Ri_inv
    Jj[:3, 3:] = np.zeros((3, 3))  # ∂t_err/∂rj = 0
    
    # Rotation 부분
    # ∂log(R_err)/∂rj = Jr^{-1} * Rij_meas^T * Ri^T
    Jj[3:, 3:] = Jr_inv @ Rij_meas.T @ Ri_inv
    
    return residual, Ji, Jj

# 테스트
res_exact, Ji_exact, Jj_exact = compute_between_jacobian_manual_exact(
    pose_i, pose_j, pose_ij_meas
)

print("📐 정확한 Jacobian 결과:")
print(f"Residual: {res_exact}")
print(f"\n근사와의 차이:")
print(f"Residual 차이: {np.linalg.norm(res_exact - res_approx):.6f}")
print(f"Ji 차이: {np.linalg.norm(Ji_exact - Ji_approx):.6f}")
print(f"Jj 차이: {np.linalg.norm(Jj_exact - Jj_approx):.6f}")

## 3. Symforce를 사용한 자동 Jacobian

이제 Symforce를 사용하여 자동으로 Jacobian을 계산해봅시다.

In [ ]:
# Symbolic 변수 정의
print("🤖 Symforce Symbolic 정의...\n")

# 회전 벡터와 이동 벡터
sf_ri = sf.V3.symbolic("ri")
sf_rj = sf.V3.symbolic("rj")
sf_rij = sf.V3.symbolic("rij")

sf_ti = sf.V3.symbolic("ti")
sf_tj = sf.V3.symbolic("tj")
sf_tij = sf.V3.symbolic("tij")

# SE(3) 변환 구성
sf_Ri = LieGroupOps.from_tangent(sf.Rot3, sf_ri)
sf_Rj = LieGroupOps.from_tangent(sf.Rot3, sf_rj)
sf_Rij = LieGroupOps.from_tangent(sf.Rot3, sf_rij)

sf_Ti = sf.Pose3(R=sf_Ri, t=sf_ti)
sf_Tj = sf.Pose3(R=sf_Rj, t=sf_tj)
sf_Tij_meas = sf.Pose3(R=sf_Rij, t=sf_tij)

# 에러 계산
sf_Tij_pred = sf_Ti.inverse() * sf_Tj
sf_T_err = sf_Tij_meas.inverse() * sf_Tij_pred
sf_err = sf.Matrix(sf_T_err.to_tangent())  # [rotation, translation] 순서

# 순서 변경 (Symforce는 [r,t], 우리는 [t,r] 사용)
sf_err_reordered = sf.Matrix.block_matrix([[sf_err[3:]], [sf_err[:3]]])

# Jacobian 계산
print("📐 Symbolic Jacobian 계산...")
sf_J_ti = sf_err_reordered.jacobian([sf_ti])
sf_J_ri = sf_err_reordered.jacobian([sf_ri])
sf_J_tj = sf_err_reordered.jacobian([sf_tj])
sf_J_rj = sf_err_reordered.jacobian([sf_rj])

print("✅ Symbolic 표현 완료!")

In [ ]:
# 코드 생성
print("🔧 최적화된 코드 생성 중...\n")

def between_error_with_jacobians(Ti: sf.Pose3, Tj: sf.Pose3, Tij_meas: sf.Pose3) -> sf.V6:
    """Between factor error with reordered output [t, r]"""
    Tij_pred = Ti.inverse() * Tj
    T_err = Tij_meas.inverse() * Tij_pred
    err = T_err.to_tangent()  # [r, t]
    # Reorder to [t, r]
    return sf.V6.block_matrix([[err[3:]], [err[:3]]])

# Codegen
between_codegen = codegen.Codegen.function(
    func=between_error_with_jacobians,
    config=codegen.PythonConfig()
)

between_with_jac = between_codegen.with_jacobians(
    which_args=["Ti", "Tj"],
    include_results=True
)

generated = between_with_jac.generate_function()

# 생성된 함수 임포트
import sys
sys.path.append(str(generated.output_dir))
from between_error_with_jacobians_with_jacobians01 import between_error_with_jacobians_with_jacobians01

print("✅ 코드 생성 완료!")

In [ ]:
def compute_between_jacobian_symforce(pose_i, pose_j, pose_ij_meas):
    """Symforce를 사용한 자동 Jacobian 계산"""
    
    # sym.Pose3 객체 생성
    Ti = sym.Pose3(
        R=sym.Rot3.from_quaternion(rotvec_to_quat(pose_i['r'])),
        t=pose_i['t']
    )
    Tj = sym.Pose3(
        R=sym.Rot3.from_quaternion(rotvec_to_quat(pose_j['r'])),
        t=pose_j['t']
    )
    Tij_meas = sym.Pose3(
        R=sym.Rot3.from_quaternion(rotvec_to_quat(pose_ij_meas['r'])),
        t=pose_ij_meas['t']
    )
    
    # 컴파일된 함수 호출
    residual, J_Ti, J_Tj = between_error_with_jacobians_with_jacobians01(
        Ti=Ti, Tj=Tj, Tij_meas=Tij_meas
    )
    
    # Jacobian 재구성 (Ti = [ti, ri], Tj = [tj, rj])
    Ji = np.zeros((6, 6))
    Ji[:, :3] = J_Ti[:, 3:]  # ∂err/∂ti
    Ji[:, 3:] = J_Ti[:, :3]  # ∂err/∂ri
    
    Jj = np.zeros((6, 6))
    Jj[:, :3] = J_Tj[:, 3:]  # ∂err/∂tj
    Jj[:, 3:] = J_Tj[:, :3]  # ∂err/∂rj
    
    return residual, Ji, Jj

# 테스트
res_symforce, Ji_symforce, Jj_symforce = compute_between_jacobian_symforce(
    pose_i, pose_j, pose_ij_meas
)

print("🤖 Symforce Jacobian 결과:")
print(f"Residual: {res_symforce}")
print(f"\n정확한 수동 계산과의 차이:")
print(f"Residual 차이: {np.linalg.norm(res_symforce - res_exact):.6f}")
print(f"Ji 차이: {np.linalg.norm(Ji_symforce - Ji_exact):.6f}")
print(f"Jj 차이: {np.linalg.norm(Jj_symforce - Jj_exact):.6f}")

## 4. 성능 비교

세 가지 방법의 성능을 비교해봅시다.

In [ ]:
# 성능 측정 함수
def benchmark_jacobian_methods(n_iterations=1000):
    """세 가지 Jacobian 계산 방법의 성능 비교"""
    
    # 테스트 데이터
    test_poses_i = []
    test_poses_j = []
    test_measurements = []
    
    np.random.seed(42)
    for _ in range(n_iterations):
        pose_i = {
            't': np.random.randn(3),
            'r': np.random.randn(3) * 0.5
        }
        pose_j = {
            't': np.random.randn(3),
            'r': np.random.randn(3) * 0.5
        }
        meas = {
            't': np.random.randn(3),
            'r': np.random.randn(3) * 0.3,
            'R': rotvec_to_rotmat(np.random.randn(3) * 0.3)
        }
        
        test_poses_i.append(pose_i)
        test_poses_j.append(pose_j)
        test_measurements.append(meas)
    
    # 근사 방법
    start = time.time()
    for i in range(n_iterations):
        compute_between_jacobian_manual_approx(
            test_poses_i[i], test_poses_j[i], test_measurements[i]
        )
    time_approx = time.time() - start
    
    # 정확한 방법
    start = time.time()
    for i in range(n_iterations):
        compute_between_jacobian_manual_exact(
            test_poses_i[i], test_poses_j[i], test_measurements[i]
        )
    time_exact = time.time() - start
    
    # Symforce 방법
    start = time.time()
    for i in range(n_iterations):
        compute_between_jacobian_symforce(
            test_poses_i[i], test_poses_j[i], test_measurements[i]
        )
    time_symforce = time.time() - start
    
    return {
        'approx': time_approx / n_iterations * 1e6,  # μs
        'exact': time_exact / n_iterations * 1e6,
        'symforce': time_symforce / n_iterations * 1e6
    }

# 벤치마크 실행
print("⏱️  성능 벤치마크 실행 중...\n")
times = benchmark_jacobian_methods(1000)

# 결과 시각화
methods = list(times.keys())
values = list(times.values())

plt.figure(figsize=(10, 6))
bars = plt.bar(methods, values, color=['red', 'green', 'blue'])
plt.ylabel('Time per iteration (μs)')
plt.title('Jacobian Computation Performance Comparison')
plt.grid(True, alpha=0.3, axis='y')

# 값 표시
for bar, value in zip(bars, values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
             f'{value:.1f} μs', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\n📊 성능 비교 결과:")
print(f"   근사 방법: {times['approx']:.1f} μs/iteration")
print(f"   정확한 방법: {times['exact']:.1f} μs/iteration")
print(f"   Symforce: {times['symforce']:.1f} μs/iteration")
print(f"\n   Symforce 속도 향상:")
print(f"   - 근사 대비: {times['approx']/times['symforce']:.1f}x")
print(f"   - 정확한 대비: {times['exact']/times['symforce']:.1f}x")

## 5. 정확도 비교

큰 회전에서 각 방법의 정확도를 비교해봅시다.

In [ ]:
def test_jacobian_accuracy(angle_range):
    """다양한 회전 크기에서 Jacobian 정확도 테스트"""
    
    errors_approx = []
    errors_exact = []
    
    for angle in angle_range:
        # 큰 회전을 가진 테스트 케이스
        pose_i = {
            't': np.array([0, 0, 0]),
            'r': np.array([0, 0, 0])
        }
        pose_j = {
            't': np.array([1, 0, 0]),
            'r': np.array([0, 0, angle])
        }
        meas = {
            't': np.array([1, 0, 0]),
            'r': np.array([0, 0, angle]),
            'R': rotvec_to_rotmat(np.array([0, 0, angle]))
        }
        
        # 각 방법으로 계산
        _, Ji_approx, Jj_approx = compute_between_jacobian_manual_approx(
            pose_i, pose_j, meas
        )
        _, Ji_exact, Jj_exact = compute_between_jacobian_manual_exact(
            pose_i, pose_j, meas
        )
        _, Ji_symforce, Jj_symforce = compute_between_jacobian_symforce(
            pose_i, pose_j, meas
        )
        
        # Symforce를 ground truth로 사용
        error_approx = np.linalg.norm(Ji_approx - Ji_symforce) + \
                      np.linalg.norm(Jj_approx - Jj_symforce)
        error_exact = np.linalg.norm(Ji_exact - Ji_symforce) + \
                     np.linalg.norm(Jj_exact - Jj_symforce)
        
        errors_approx.append(error_approx)
        errors_exact.append(error_exact)
    
    return errors_approx, errors_exact

# 테스트 실행
angles = np.linspace(0, np.pi, 50)
errors_approx, errors_exact = test_jacobian_accuracy(angles)

# 시각화
plt.figure(figsize=(10, 6))
plt.semilogy(np.rad2deg(angles), errors_approx, 'r-', label='Approximate Method', linewidth=2)
plt.semilogy(np.rad2deg(angles), errors_exact, 'g-', label='Exact Method', linewidth=2)
plt.xlabel('Rotation Angle (degrees)')
plt.ylabel('Jacobian Error (log scale)')
plt.title('Jacobian Accuracy vs Rotation Magnitude')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("📊 정확도 분석:")
print(f"   작은 각도 (< 10°)에서:")
small_idx = angles < np.deg2rad(10)
print(f"   - 근사 평균 오차: {np.mean(np.array(errors_approx)[small_idx]):.6f}")
print(f"   - 정확한 평균 오차: {np.mean(np.array(errors_exact)[small_idx]):.6f}")
print(f"\n   큰 각도 (> 90°)에서:")
large_idx = angles > np.deg2rad(90)
print(f"   - 근사 평균 오차: {np.mean(np.array(errors_approx)[large_idx]):.6f}")
print(f"   - 정확한 평균 오차: {np.mean(np.array(errors_exact)[large_idx]):.6f}")

## 6. 최적화에 미치는 영향

다른 Jacobian이 최적화 결과에 어떤 영향을 미치는지 확인해봅시다.

In [ ]:
def create_test_problem():
    """테스트용 pose graph 문제 생성"""
    poses = {}
    edges = []
    
    # 원형 경로 (큰 회전 포함)
    n_poses = 8
    radius = 2.0
    
    for i in range(n_poses):
        angle = 2 * np.pi * i / n_poses
        x = radius * np.cos(angle)
        y = radius * np.sin(angle)
        theta = angle + np.pi/2  # 접선 방향
        
        # 노이즈 추가
        poses[i] = {
            't': np.array([x + np.random.randn()*0.1, 
                          y + np.random.randn()*0.1, 0]),
            'r': np.array([0, 0, theta + np.random.randn()*0.1])
        }
    
    # Odometry edges
    for i in range(n_poses):
        j = (i + 1) % n_poses
        
        # 상대 변환 (큰 회전)
        dtheta = 2 * np.pi / n_poses
        edges.append({
            'from': i,
            'to': j,
            't': np.array([2*radius*np.sin(dtheta/2), 0, 0]),
            'r': np.array([0, 0, dtheta]),
            'R': rotvec_to_rotmat(np.array([0, 0, dtheta])),
            'information': np.eye(6) * 100
        })
    
    # Loop closure
    edges.append({
        'from': 0,
        'to': n_poses//2,
        't': np.array([0, 2*radius, 0]),
        'r': np.array([0, 0, np.pi]),
        'R': rotvec_to_rotmat(np.array([0, 0, np.pi])),
        'information': np.eye(6) * 50
    })
    
    return poses, edges

# 간단한 optimizer (Chapter 4에서 가져옴)
def optimize_with_jacobian(poses, edges, jacobian_func, max_iters=20):
    """특정 Jacobian 함수를 사용한 최적화"""
    poses_copy = {k: {'t': v['t'].copy(), 'r': v['r'].copy()} 
                  for k, v in poses.items()}
    errors = []
    
    for iteration in range(max_iters):
        # 총 에러 계산
        total_error = 0
        
        # H와 b 구축 (간소화)
        n_poses = len(poses_copy)
        H = np.zeros((n_poses*6, n_poses*6))
        b = np.zeros(n_poses*6)
        
        for edge in edges:
            i = edge['from']
            j = edge['to']
            
            residual, Ji, Jj = jacobian_func(
                poses_copy[i], poses_copy[j], edge
            )
            
            omega = edge['information']
            total_error += residual.T @ omega @ residual
            
            # H와 b 업데이트
            H[i*6:(i+1)*6, i*6:(i+1)*6] += Ji.T @ omega @ Ji
            H[j*6:(j+1)*6, j*6:(j+1)*6] += Jj.T @ omega @ Jj
            H[i*6:(i+1)*6, j*6:(j+1)*6] += Ji.T @ omega @ Jj
            H[j*6:(j+1)*6, i*6:(i+1)*6] += Jj.T @ omega @ Ji
            
            b[i*6:(i+1)*6] += Ji.T @ omega @ residual
            b[j*6:(j+1)*6] += Jj.T @ omega @ residual
        
        errors.append(total_error)
        
        # 첫 번째 포즈 고정
        H[0:6, 0:6] += np.eye(6) * 1e10
        
        # 선형 시스템 해결
        try:
            dx = np.linalg.solve(H, -b)
        except:
            break
            
        # 업데이트
        for k in range(n_poses):
            poses_copy[k]['t'] += dx[k*6:k*6+3]
            poses_copy[k]['r'] += dx[k*6+3:k*6+6]
            
        if np.linalg.norm(dx) < 1e-6:
            break
    
    return poses_copy, errors

# 테스트 실행
print("🔨 테스트 문제 생성 및 최적화...\n")
poses_init, edges = create_test_problem()

# 각 방법으로 최적화
poses_approx, errors_approx = optimize_with_jacobian(
    poses_init, edges, compute_between_jacobian_manual_approx
)
poses_exact, errors_exact = optimize_with_jacobian(
    poses_init, edges, compute_between_jacobian_manual_exact
)
poses_symforce, errors_symforce = optimize_with_jacobian(
    poses_init, edges, compute_between_jacobian_symforce
)

# 수렴 곡선 비교
plt.figure(figsize=(10, 6))
plt.semilogy(errors_approx, 'r-o', label='Approximate Jacobian')
plt.semilogy(errors_exact, 'g-o', label='Exact Manual Jacobian')
plt.semilogy(errors_symforce, 'b-o', label='Symforce Jacobian')
plt.xlabel('Iteration')
plt.ylabel('Total Error (log scale)')
plt.title('Optimization Convergence with Different Jacobians')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("📊 수렴 분석:")
print(f"   최종 에러:")
print(f"   - 근사 Jacobian: {errors_approx[-1]:.6f}")
print(f"   - 정확한 Jacobian: {errors_exact[-1]:.6f}")
print(f"   - Symforce: {errors_symforce[-1]:.6f}")
print(f"\n   수렴 속도 (90% 에러 감소까지):")
target_error = 0.1 * errors_approx[0]
conv_approx = next((i for i, e in enumerate(errors_approx) if e < target_error), len(errors_approx))
conv_exact = next((i for i, e in enumerate(errors_exact) if e < target_error), len(errors_exact))
conv_symforce = next((i for i, e in enumerate(errors_symforce) if e < target_error), len(errors_symforce))
print(f"   - 근사: {conv_approx} iterations")
print(f"   - 정확한: {conv_exact} iterations")
print(f"   - Symforce: {conv_symforce} iterations")

## 7. 실전 팁: 수치적 미분 검증

Jacobian이 올바른지 수치적 미분으로 검증하는 방법입니다.

In [ ]:
def numerical_jacobian(func, x, epsilon=1e-7):
    """수치적 미분으로 Jacobian 계산"""
    f0 = func(x)
    n_out = len(f0)
    n_in = len(x)
    J = np.zeros((n_out, n_in))
    
    for i in range(n_in):
        x_plus = x.copy()
        x_plus[i] += epsilon
        f_plus = func(x_plus)
        
        x_minus = x.copy()
        x_minus[i] -= epsilon
        f_minus = func(x_minus)
        
        J[:, i] = (f_plus - f_minus) / (2 * epsilon)
    
    return J

def verify_jacobian(jacobian_func):
    """Jacobian 함수의 정확성 검증"""
    # 테스트 데이터
    pose_i = {'t': np.array([1.0, 0.5, 0.2]), 'r': np.array([0.1, 0.05, 0.15])}
    pose_j = {'t': np.array([2.0, 1.0, 0.3]), 'r': np.array([0.2, 0.1, 0.3])}
    meas = {
        't': np.array([1.0, 0.5, 0.1]),
        'r': np.array([0.1, 0.05, 0.15]),
        'R': rotvec_to_rotmat(np.array([0.1, 0.05, 0.15]))
    }
    
    # 분석적 Jacobian
    residual, Ji_analytical, Jj_analytical = jacobian_func(pose_i, pose_j, meas)
    
    # 수치적 Jacobian - pose i
    def residual_func_i(x):
        pose_i_mod = {
            't': x[:3],
            'r': x[3:]
        }
        res, _, _ = jacobian_func(pose_i_mod, pose_j, meas)
        return res
    
    x_i = np.concatenate([pose_i['t'], pose_i['r']])
    Ji_numerical = numerical_jacobian(residual_func_i, x_i)
    
    # 수치적 Jacobian - pose j
    def residual_func_j(x):
        pose_j_mod = {
            't': x[:3],
            'r': x[3:]
        }
        res, _, _ = jacobian_func(pose_i, pose_j_mod, meas)
        return res
    
    x_j = np.concatenate([pose_j['t'], pose_j['r']])
    Jj_numerical = numerical_jacobian(residual_func_j, x_j)
    
    # 비교
    error_Ji = np.linalg.norm(Ji_analytical - Ji_numerical)
    error_Jj = np.linalg.norm(Jj_analytical - Jj_numerical)
    
    return error_Ji, error_Jj, Ji_analytical, Ji_numerical

# 각 방법 검증
print("🔍 Jacobian 수치 검증:\n")

methods = [
    ("근사 방법", compute_between_jacobian_manual_approx),
    ("정확한 방법", compute_between_jacobian_manual_exact),
    ("Symforce", compute_between_jacobian_symforce)
]

for name, func in methods:
    error_Ji, error_Jj, Ji_ana, Ji_num = verify_jacobian(func)
    print(f"{name}:")
    print(f"   Ji 오차: {error_Ji:.6e}")
    print(f"   Jj 오차: {error_Jj:.6e}")
    print(f"   {'✅ 통과' if error_Ji < 1e-5 and error_Jj < 1e-5 else '❌ 실패'}\n")

# 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# 마지막 테스트의 Ji 비교
im1 = ax1.imshow(np.abs(Ji_ana - Ji_num), cmap='hot', interpolation='nearest')
ax1.set_title('|Analytical - Numerical| Jacobian Ji')
ax1.set_xlabel('Input dimension')
ax1.set_ylabel('Output dimension')
plt.colorbar(im1, ax=ax1)

# 상대 오차
relative_error = np.abs(Ji_ana - Ji_num) / (np.abs(Ji_ana) + 1e-10)
im2 = ax2.imshow(relative_error, cmap='hot', interpolation='nearest')
ax2.set_title('Relative Error in Jacobian Ji')
ax2.set_xlabel('Input dimension')
ax2.set_ylabel('Output dimension')
plt.colorbar(im2, ax=ax2)

plt.tight_layout()
plt.show()

## 8. 요약 및 핵심 포인트

### 🎓 이 챕터에서 배운 내용:

1. **Jacobian 계산 방법들**
   - **근사 방법**: 빠르지만 큰 회전에서 부정확
   - **정확한 수동 방법**: Lie theory 사용, 복잡함
   - **Symforce 자동 방법**: 빠르고 정확함

2. **성능 비교**
   - Symforce가 수동 방법보다 빠름
   - 컴파일된 코드의 효율성
   - 병렬화 가능성

3. **정확도의 중요성**
   - 큰 회전에서 근사의 한계
   - 수렴 속도와 안정성에 영향
   - 최종 정확도 향상

4. **실전 팁**
   - 수치적 미분으로 검증
   - 문제에 맞는 방법 선택
   - 디버깅 기법

### 💡 다음 챕터 예고:

다음 챕터에서는 Robust optimization을 위한 Cauchy kernel과 다른 robust loss function들을 다룹니다.

## 🏋️ 연습 문제

### 문제 1: 다른 에러 표현
현재는 $T_{ij}^{-1} \cdot T_i^{-1} \cdot T_j$ 형태의 에러를 사용합니다. $T_i \cdot T_{ij} \cdot T_j^{-1}$ 형태의 에러에 대한 Jacobian을 유도해보세요.

### 문제 2: SE(2) Jacobian
2D SLAM을 위한 SE(2) between factor의 정확한 Jacobian을 구현해보세요.

### 문제 3: 적응형 방법 선택
회전 크기에 따라 자동으로 근사/정확한 방법을 선택하는 적응형 Jacobian 계산기를 만들어보세요.

In [ ]:
# 여기에 연습 문제를 풀어보세요!

# 문제 3 예시: 적응형 Jacobian
def compute_between_jacobian_adaptive(pose_i, pose_j, pose_ij_meas, 
                                     angle_threshold=np.deg2rad(10)):
    """회전 크기에 따라 방법을 선택하는 적응형 Jacobian"""
    # 회전 크기 계산
    # 여기에 구현
    pass
